<a href="https://colab.research.google.com/github/jaeyoung-jae-park/Joint-model-assisted-Decision-Rule/blob/main/Numerical_experiment_Toxic_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data preparation

We mount google drive and load the dataset from Kaggle. If you 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jaeyoungpark","key":"0560c108cef0bdd5c48cc2c16db24253"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

  0% 0.00/1.46M [00:00<?, ?B/s]
100% 1.46M/1.46M [00:00<00:00, 99.2MB/s]
  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 93.6MB/s]
 73% 17.0M/23.4M [00:00<00:00, 30.3MB/s]
100% 23.4M/23.4M [00:00<00:00, 59.3MB/s]
 34% 9.00M/26.3M [00:00<00:00, 27.4MB/s]
100% 26.3M/26.3M [00:00<00:00, 59.4MB/s]


In [ ]:
!ls

drive	     sample_data		test.csv.zip	     train.csv.zip
kaggle.json  sample_submission.csv.zip	test_labels.csv.zip


In [ ]:
!mkdir toxic
!unzip sample_submission.csv.zip -d toxic
!unzip test_labels.csv.zip -d toxic
!unzip test.csv.zip -d toxic
!unzip train.csv.zip -d toxic

Archive:  sample_submission.csv.zip
  inflating: toxic/sample_submission.csv  
Archive:  test_labels.csv.zip
  inflating: toxic/test_labels.csv   
Archive:  test.csv.zip
  inflating: toxic/test.csv          
Archive:  train.csv.zip
  inflating: toxic/train.csv         


In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv("/content/toxic/train.csv")
# test = pd.read_csv("/content/toxic/test.csv")

In [ ]:
np.hstack((train['toxic'].value_counts()[1], 
           train['severe_toxic'].value_counts()[1], 
           train['obscene'].value_counts()[1],
           train['threat'].value_counts()[1],
           train['insult'].value_counts()[1],
           train['identity_hate'].value_counts()[1]))

array([15294,  1595,  8449,   478,  7877,  1405])

In [ ]:
train['toxic'].value_counts()/np.sum(train['toxic'].value_counts())

0    0.904156
1    0.095844
Name: toxic, dtype: float64

In [ ]:
train['severe_toxic'].value_counts()/np.sum(train['severe_toxic'].value_counts())

0    0.990004
1    0.009996
Name: severe_toxic, dtype: float64

In [ ]:
train['obscene'].value_counts()/np.sum(train['obscene'].value_counts())

0    0.947052
1    0.052948
Name: obscene, dtype: float64

In [ ]:
train['threat'].value_counts()/np.sum(train['threat'].value_counts())

0    0.997004
1    0.002996
Name: threat, dtype: float64

In [ ]:
train['insult'].value_counts()/np.sum(train['insult'].value_counts())

0    0.950636
1    0.049364
Name: insult, dtype: float64

In [ ]:
train['identity_hate'].value_counts()/np.sum(train['identity_hate'].value_counts())

0    0.991195
1    0.008805
Name: identity_hate, dtype: float64

In [ ]:
pd.crosstab(train['toxic'], train['obscene'])

obscene,0,1
toxic,,
0,143754,523
1,7368,7926


In [ ]:
pd.crosstab(train['toxic'], train['insult'])

insult,0,1
toxic,,
0,143744,533
1,7950,7344


In [ ]:
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


7926

In [ ]:
X = train['comment_text']
y = train[['toxic','severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]
y_aux = train[['toxic', 'obscene', 'insult']]
# y_aux1 = train[['toxic', 'obscene']]
# y_aux2 = train[['toxic', 'insult']]
y_aux_agg = pd.concat([y_aux, y_aux.iloc[:,0] * y_aux.iloc[:,1],
                        y_aux.iloc[:,0] * y_aux.iloc[:,2]], axis=1)
# y_aux1_agg = pd.concat([y_aux1, y_aux1.iloc[:,0] * y_aux1.iloc[:,1]], axis=1)
# y_aux2_agg = pd.concat([y_aux2, y_aux2.iloc[:,0] * y_aux2.iloc[:,1]], axis=1)
y_tgt = train['toxic']
# y_tgt = train['obscene']
# y = train['insult']

In [ ]:
y_aux_agg

,toxic,obscene,insult,0,1
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
159566,0,0,0,0,0
159567,0,0,0,0,0
159568,0,0,0,0,0
159569,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_tgt, test_size = 0.1, random_state=1, shuffle=True)

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

vocab_size = 5000

tokenizer_obj = Tokenizer(num_words = vocab_size)
tokenizer_obj.fit_on_texts(X_train)

max_length = max([len(s.split()) for s in X])

# vocab_size = len(tokenizer_obj.word_index) + 1

X_train_tokens = tokenizer_obj.texts_to_matrix(X_train, mode='tfidf')
X_test_tokens = tokenizer_obj.texts_to_matrix(X_test, mode='tfidf')


# X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)
# X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)

# X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
# X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')

NameError: ignored

In [ ]:
X_train_tokens.shape

(143613, 5000)

In [ ]:
y_train

50710     0
70206     0
45716     0
93725     0
74710     0
         ..
73349     0
109259    0
50057     0
5192      0
128037    1
Name: toxic, Length: 143613, dtype: int64

# Vanilla

In [ ]:
y_aux1.shape

(159571, 2)

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Dropout
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import SGD


def define_Vanilla(output_dim):

  output_dim = output_dim

  model = Sequential()
  model.add(Dense(256, input_shape=(vocab_size,), activation='relu'))
  # model.add(Dropout(0.3))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.1))
  # model.add(Dense(1024, activation='relu'))
  model.add(Dense(output_dim, activation = "sigmoid"))
  model.summary()
  return model

def train_Vanilla(model, X, y, epochs, batch_size):
  if model is None:
    if len(y.shape) == 1:
      model = define_Vanilla(1)
    else: model= define_Vanilla(y.shape[1])
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
  rl = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
  # model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
  # model.compile(loss='binary_crossentropy', optimizer='ada  grad', metrics=['accuracy'])
  model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate = 0.001, momentum=0.9), metrics=['accuracy'])
  # model.fit(X_train_tokens , y_aux_train, batch_size = 128, epochs = 50, validation_split=0.2, callbacks=[es])
  history = model.fit(X , y, batch_size = batch_size, epochs = epochs, validation_split=0.2, callbacks=[es, rl], verbose=2)
  # history = model.fit(X , y, batch_size = batch_size, epochs = epochs, validation_split=0.2)
  return model, history


In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Dropout
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping

def define_Vanilla_linear(output_dim):

  output_dim = output_dim

  model = Sequential()
  model.add(Dense(output_dim, input_shape=(vocab_size,), activation = "sigmoid"))
  model.summary()
  return model

def train_Vanilla_linear(model, X, y, epochs, batch_size):
  if model is None:
    if len(y.shape) == 1:
      model = define_Vanilla_linear(1)
    else: model= define_Vanilla_linear(y.shape[1])
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
  rl = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

  # model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
  model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate = 0.001, momentum=0.9), metrics=['accuracy'])
  # model.fit(X_train_tokens , y_aux_train, batch_size = 128, epochs = 50, validation_split=0.2, callbacks=[es])
  history = model.fit(X , y, batch_size = batch_size, epochs = epochs, validation_split=0.2, callbacks=[es, rl], verbose = 2)
  # history = model.fit(X , y, batch_size = batch_size, epochs = epochs, validation_split=0.2)
  return model, history


In [ ]:
model, history = train_Vanilla(None, X_train_tokens, y_train, epochs=300, batch_size=128)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               1280256   
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 1,313,281
Trainable params: 1,313,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
898/898 [==============================] - 6s 4ms/step - loss: 0.4066 - accuracy: 0.8849 - val_loss: 0.3021 - val_accuracy: 0.9065
Epoch 2/300
898/898 [==============================] - 3s 4ms/step - loss: 0.2960 - accuracy: 0.904

In [ ]:
pd.crosstab(y_test, ((model.predict(X_test_tokens) > 0.5)*1).reshape(-1))

col_0,0,1
insult,,
0,15011,143
1,331,473


In [ ]:
(14993+491)/(14213+1038+177+530)

0.9702970297029703

In [ ]:
(14985+609)/(14213+1038+177+530)

0.9771901240756987

In [ ]:
pd.crosstab(y_test, ((model.predict(X_test_tokens) > 0.5)*1).reshape(-1))

col_0,0,1
toxic,,
0,14221,169
1,521,1047


In [ ]:
(14225+1051)/(14213+1038+177+530)

0.9572628148890838

In [ ]:
pd.crosstab(y_test, ((model.predict(X_test_tokens) > 0.5)*1).reshape(-1))

col_0,0,1
toxic,,
0,14168,222
1,459,1109


# Auxiliary models

In [ ]:
import tensorflow as tf
# sample_size = y_aux1_train.shape[0]
# CI_denom = (tf.square(tf.reduce_sum(y_aux1_train[:,2], axis=-1)/sample_size - 
#                       tf.reduce_sum(y_aux1_train[:,1], axis=-1)/sample_size * 
#                       tf.reduce_sum(y_aux1_train[:,0], axis=-1)/sample_size))

def CIScore(y_true, y_pred):
  CI_numer = []
  CI_numer.append(tf.square((y_pred[:, 3] - y_pred[:, 1] * y_pred[:, 0])))
  CI_numer.append(tf.square((y_pred[:, 4] - y_pred[:, 2] * y_pred[:, 0])))
  # tf.dtypes.cast(CI_numer, dtype=tf.float64)
  # CI_denom = tf.square((tf.reduce_mean(y_true[:, 2], axis=-1) - tf.reduce_mean(y_true[:, 1], axis=-1)* tf.reduce_mean(y_true[:, 0], axis=-1) ))  
  
  # tf.dtypes.cast(CI_denom, dtype=tf.float64)
  return tf.reduce_mean(CI_numer[0], axis=-1)/CI_denom[0] + tf.reduce_mean(CI_numer[1], axis=-1)/CI_denom[1] + tf.keras.losses.binary_crossentropy(y_true[:,0], y_pred[:,0])
  # return tf.divide(tf.reduce_mean(CI_numer, axis=-1),CI_denom) + tf.keras.losses.binary_crossentropy(y_true, y_pred)
  # return tf.reduce_mean(CI_numer/CI_denom, axis=-1) + tf.keras.losses.binary_crossentropy(y_true, y_pred)
  

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Dropout
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping

def define_Auxiliary(output_dim):

  output_dim = output_dim

  model = Sequential()
  model.add(Dense(256, input_shape=(vocab_size,), activation='relu'))
  # model.add(Dropout(0.3))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(output_dim, activation = "sigmoid"))
  model.summary()
  return model

def train_Auxiliary(model, X, y, epochs, batch_size):
  if model is None:
    model = define_Auxiliary(y.shape[1])

  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
  rl = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

  # model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
  model.compile(loss=CIScore, optimizer=SGD(learning_rate = 0.001, momentum=0.9), metrics=['accuracy'])
  # model.fit(X_train_tokens , y_aux_train, batch_size = 128, epochs = 50, validation_split=0.2, callbacks=[es])
  history = model.fit(X , y, batch_size = batch_size, epochs = epochs, validation_split=0.2, callbacks=[es, rl], verbose=2)
  # history = model.fit(X , y, batch_size = batch_size, epochs = epochs, validation_split=0.2)
  return model, history

def train_Auxiliary_ce_loss(model, X, y, epochs, batch_size):
  if model is None:
    model = define_Auxiliary(y.shape[1])

  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
  rl = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

  # model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
  # model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['accuracy'])
  model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate = 0.001, momentum=0.9), metrics=['accuracy'])
  
  # model.fit(X_train_tokens , y_aux_train, batch_size = 128, epochs = 50, validation_split=0.2, callbacks=[es])
  history = model.fit(X , y, batch_size = batch_size, epochs = epochs, validation_split=0.2, callbacks=[es, rl], verbose=2)
  # history = model.fit(X , y, batch_size = batch_size, epochs = epochs, validation_split=0.2)
  return model, history



In [ ]:
model, history = train_Auxiliary(model=None, X = X_train_tokens, y = y_aux1_train, epochs=50, batch_size=128)

NameError: ignored

In [ ]:
predicted_values2 = model.predict(X_test_tokens)

In [ ]:
np.mean((predicted_values2 > 0.5)*1 == y_aux1_test)

0.9540640471266528

In [ ]:
np.sum((predicted_values2 > 0.5)*1)

552

In [ ]:
np.sum(y_aux1_test)

3072

In [ ]:
from keras.models import Sequential, Model

extraction1 = Model(inputs=model.inputs, outputs=model.layers[-2].output)
extracted_features1 = extraction1.predict(X_train_tokens)

# Transfer learning

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Dropout
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping

def define_Transfer(input_shape):

  output_dim = 1

  model = Sequential()
  model.add(Dense(256, input_shape=(input_shape,), activation='relu'))
  # model.add(Dropout(0.3))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(output_dim, activation = "sigmoid"))
  # model.add(Dropout(0.1), input_shape=(input_shape,))
  # model.add(Dense(output_dim, input_dim = input_shape, activation = "sigmoid"))
  model.summary()
  return model

def train_Transfer(model, X, X_extracted, y, epochs, batch_size):
  X_transfered = np.hstack((X, X_extracted))

  if model is None:
    model = define_Transfer(X_transfered.shape[1])

  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
  rl = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

  # model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
  # model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['accuracy'])
  model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate = 0.001, momentum=0.9), metrics=['accuracy'])

  
  # model.fit(X_train_tokens , y_aux_train, batch_size = 128, epochs = 50, validation_split=0.2, callbacks=[es])
  history = model.fit(X_transfered , y, batch_size = batch_size, epochs = epochs, validation_split=0.2, callbacks=[es, rl], verbose = 2)
  # history = model.fit(X , y, batch_size = batch_size, epochs = epochs, validation_split=0.2)
  return model, history


In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Dropout
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers


def define_Transfer_linear(input_shape):

  output_dim = 1

  model = Sequential()
  model.add(Dense(output_dim, input_dim = input_shape, activation = "sigmoid"))
  model.summary()
  return model

def train_Transfer_linear(model, X, X_extracted, y, epochs, batch_size):
  X_transfered = np.hstack((X, X_extracted))

  if model is None:
    model = define_Transfer_linear(X_transfered.shape[1])

  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
  rl = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

  # model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
  # model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['accuracy'])
  model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate = 0.001, momentum=0.9), metrics=['accuracy'])

  # model.fit(X_train_tokens , y_aux_train, batch_size = 128, epochs = 50, validation_split=0.2, callbacks=[es])
  history = model.fit(X_transfered , y, batch_size = batch_size, epochs = epochs, validation_split=0.2, callbacks=[es,rl], verbose = 2)
  # history = model.fit(X , y, batch_size = batch_size, epochs = epochs, validation_split=0.2)
  return model, history


In [ ]:
model_trasnf = train_Transfer(model=None, X = X_train_tokens, X_extracted = extracted_features1, y = y_tgt_train, epochs=300, batch_size=128)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               1313024   
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 1,346,049
Trainable params: 1,346,049
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
898/898 [==============================] - 4s 4ms/step - loss: 0.3034 - accuracy: 0.8889 - val_loss: 0.1831 - val_accuracy: 0.9303
Epoch 2/300
898/898 [==============================] - 3s 4ms/step - loss: 0.1589 - accuracy: 0.9399 - val_loss: 0.1513 - val_accuracy: 0.9481
Epoch 3/300
898/898 [==============================] - 3s 4ms/step - loss: 0.1252 - accu

# Experiments

In [ ]:
X = np.array(X)
y_aux_agg = np.array(y_aux_agg)
# y_aux1_agg = np.array(y_aux1_agg)
# y_aux2_agg = np.array(y_aux2_agg)
y_tgt = np.array(y_tgt)
# y_aux1 = np.array(y_aux1)
y_aux = np.array(y_aux)
np.random.seed(42)
fold_idx = np.random.choice(np.hstack((np.repeat(np.arange(10), int(X.shape[0]/10)), np.arange(X.shape[0] % 10))), size=X.shape[0], replace=False)

def split_data(X, y, fold_no):
  X_train, X_test, y_train, y_test = X[fold_idx != fold_no], X[fold_idx == fold_no], y[fold_idx != fold_no], y[fold_idx == fold_no]
  np.random.seed(fold_no)
  shuffle_train = np.random.choice(X_train.shape[0], X_train.shape[0], replace=False)
  shuffle_test = np.random.choice(X_test.shape[0], X_test.shape[0], replace=False)
  return X_train[shuffle_train], X_test[shuffle_test], y_train[shuffle_train], y_test[shuffle_test]

# X_train, X_test, y_tgt_train, y_tgt_test = train_test_split(X, y_tgt, test_size = 0.1, random_state=1, shuffle=True)

In [ ]:
pd.DataFrame(fold_idx).value_counts()

0    15958
9    15957
8    15957
7    15957
6    15957
5    15957
4    15957
3    15957
2    15957
1    15957
dtype: int64

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
import keras

for fold_no in np.arange(1):
  X_train, X_test, y_aux1_train, y_aux1_test = split_data(X, y_aux_agg, fold_no)
  # X_train, X_test, y_aux1_train, y_aux1_test = split_data(X, y_aux1_agg, fold_no)
  # X_train, X_test, y_aux2_train, y_aux2_test = split_data(X, y_aux2_agg, fold_no)
  X_train, X_test, y_tgt_train, y_tgt_test = split_data(X, y_tgt, fold_no)
  # X_train, X_test, y_joint_train, y_joint_test = split_data(X, y_aux1, fold_no)
  y_aux1_train = np.array(y_aux1_train, dtype='float32')

  vocab_size = 5000

  tokenizer_obj = Tokenizer(num_words = vocab_size)
  tokenizer_obj.fit_on_texts(X_train)

  max_length = max([len(s.split()) for s in X])

  X_train_tokens = tokenizer_obj.texts_to_matrix(X_train, mode='tfidf')
  X_test_tokens = tokenizer_obj.texts_to_matrix(X_test, mode='tfidf')

  # model_Vanilla, history_Vanilla = train_Vanilla(model = None, X= X_train_tokens, y = y_tgt_train, epochs = 300, batch_size = 256)
  # model_Vanilla.save("/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_Vanilla_"+str(fold_no))
  # del model_Vanilla
  # keras.backend.clear_session()

  # model_linear, history_linear = train_Vanilla_linear(model = None, X= X_train_tokens, y = y_tgt_train, epochs = 300, batch_size = 256)
  # model_linear.save("/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_Vanilla_linear_"+str(fold_no))
  # del model_linear
  # keras.backend.clear_session()

  # model_joint, history_joint = train_Vanilla(model = None, X= X_train_tokens, y = y_aux1_train[:,0:3], epochs = 300, batch_size = 256)
  # model_joint.save("/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_Vanilla_joint_"+str(fold_no))
  # del model_joint
  # keras.backend.clear_session()


  # # Auxiliary 1

  sample_size = y_aux1_train.shape[0]
  CI_denom = []
  CI_denom.append((tf.square(tf.reduce_sum(y_aux1_train[:,3], axis=-1)/sample_size - 
                          tf.reduce_sum(y_aux1_train[:,1], axis=-1)/sample_size * 
                          tf.reduce_sum(y_aux1_train[:,0], axis=-1)/sample_size)))
  CI_denom.append((tf.square(tf.reduce_sum(y_aux1_train[:,4], axis=-1)/sample_size - 
                          tf.reduce_sum(y_aux1_train[:,2], axis=-1)/sample_size * 
                          tf.reduce_sum(y_aux1_train[:,0], axis=-1)/sample_size)))

  model_Aux1, history_Aux1 = train_Auxiliary(model = None, X = X_train_tokens, y = y_aux1_train, epochs=300, batch_size = 256)
  # model_Aux1.save("/content/drive/MyDrive/Transfer_learning/Toxic_SGD2/model_Aux1_"+str(fold_no))
  # del model_Aux1


  # CIDNN
  # model_Aux1 = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic_SGD2/model_Aux1_'+str(fold_no), compile=False)
  extraction1 = Model(inputs=model_Aux1.inputs, outputs=model_Aux1.layers[-2].output)
  extracted_features1 = extraction1.predict(X_train_tokens)
  del model_Aux1, extraction1
  keras.backend.clear_session()
  
  model_transf_linear1, history_transf_linear1 = train_Transfer_linear(model = None, X= X_train_tokens, X_extracted = extracted_features1, y = y_tgt_train, epochs= 300, batch_size = 256)
  # model_transf_linear1.save("/content/drive/MyDrive/Transfer_learning/Toxic_SGD2/model_transf_linear1_"+str(fold_no))
  del model_transf_linear1
  keras.backend.clear_session()

  model_transf1, history_transf1 = train_Transfer(model = None, X= X_train_tokens, X_extracted = extracted_features1, y = y_tgt_train, epochs= 300, batch_size = 256)
  # model_transf1.save("/content/drive/MyDrive/Transfer_learning/Toxic_SGD2/model_transf1_"+str(fold_no))
  del model_transf1, extracted_features1
  keras.backend.clear_session()

  
  # Conventional
  # model_Aux_ce1 = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_Vanilla_joint_'+str(fold_no), compile=False)
  # extraction_ce1 = Model(inputs=model_Aux_ce1.inputs, outputs=model_Aux_ce1.layers[-2].output)
  # extracted_features_ce1 = extraction_ce1.predict(X_train_tokens)
  # del model_Aux_ce1, extraction_ce1
  # keras.backend.clear_session()

  # model_transf_ce_linear1, history_transf_ce_linear1 = train_Transfer_linear(model = None, X= X_train_tokens, X_extracted = extracted_features_ce1, y = y_tgt_train, epochs= 300, batch_size = 256)
  # model_transf_ce_linear1.save("/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_transf_ce_linear1_revised_"+str(fold_no))
  # del model_transf_ce_linear1
  # keras.backend.clear_session()

  # model_transf_ce1, history_transf_ce1 = train_Transfer(model = None, X = X_train_tokens, X_extracted = extracted_features_ce1, y = y_tgt_train, epochs = 300, batch_size = 256)
  # model_transf_ce1.save("/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_transf_ce1_revised_"+str(fold_no))
  # del model_transf_ce1
  # keras.backend.clear_session()

 

  # del X_train, X_test, y_aux1_train, y_aux1_test, y_aux2_train, y_aux2_test, y_tgt_train, y_tgt_test, tokenizer_obj, X_train_tokens, X_test_tokens
  del X_train, X_test, y_aux1_train, y_aux1_test, y_tgt_train, y_tgt_test, tokenizer_obj, X_train_tokens, X_test_tokens

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               1280256   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
Total params: 1,313,797
Trainable params: 1,313,797
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
449/449 - 3s - loss: 1.6879 - accuracy: 0.9162 - val_loss: 0.4152 - val_accuracy: 0.8336
Epoch 2/300
449/449 - 2s - loss: 0.4218 - accuracy: 0.6186 - val_loss: 0.3697 - val_accuracy: 0.5443
Epoch 3/300
449/449 - 2s - loss: 0.3869 - 

In [ ]:
extracted_features2.shape

(15958, 128)

In [ ]:
model_Vanilla = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_Vanilla_'+str(fold_no), compile=False)
predicted_vanilla = model_Vanilla.predict(X_test_tokens)

In [ ]:
pd.crosstab(y_tgt_test, 
            ((predicted_vanilla>0.5)*1).reshape(-1))

col_0,0,1
row_0,,
0,14280,169
1,511,998


In [ ]:
np.mean(y_tgt_test == ((predicted_vanilla>0.5)*1).reshape(-1))

0.9573881438776789

In [ ]:
model_Aux1 = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_Aux2_'+str(fold_no), compile=False)
extraction1 = Model(inputs=model_Aux1.inputs, outputs=model_Aux1.layers[-2].output)
extracted_features1 = extraction1.predict(X_test_tokens)


In [ ]:
model_transf1 = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_transf2_'+str(fold_no), compile=False)
predicted_values = model_transf1.predict(np.hstack((X_test_tokens, extracted_features1)))

In [ ]:
pd.crosstab(y_tgt_test, 
            ((predicted_values>0.5)*1).reshape(-1))

col_0,0,1
row_0,,
0,14267,182
1,490,1019


In [ ]:
np.mean(y_tgt_test == ((predicted_values>0.5)*1).reshape(-1))

0.9578894598320592

In [ ]:
X_test_tokens.shape

(15958, 5000)

In [ ]:
y_aux1_test.shape

(15957, 3)

In [ ]:
X_train_tokens.shape

(143613, 5000)

In [ ]:
y_aux1_train.shape

(143614, 3)

In [ ]:
y_aux1_test

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int64)

In [ ]:
((predicted_values > 0.5)*1).shape

(15958, 3)

In [ ]:
y_aux1_test.shape

(15957, 3)

In [ ]:
extracted_features1

array([[0.35000652, 0.        , 0.        , ..., 0.        , 0.        ,
        0.32276964],
       [0.173515  , 0.        , 0.        , ..., 0.        , 0.        ,
        0.35561523],
       [0.26332697, 0.        , 0.        , ..., 0.        , 0.        ,
        0.6739029 ],
       ...,
       [0.        , 0.01729007, 0.        , ..., 0.10691443, 0.01313869,
        0.40789366],
       [0.        , 0.        , 0.        , ..., 0.31632534, 0.30796963,
        0.502854  ],
       [0.        , 0.25563458, 0.        , ..., 0.05105225, 0.18806519,
        0.7389372 ]], dtype=float32)

In [ ]:
keras.backe

In [ ]:
keras.backend.clear_session()

In [ ]:
!nvidia-smi

Sun May  9 20:19:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    38W / 250W |    929MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
predicted_values = model_Vanilla.predict(X_test_tokens)

ValueError: ignored

In [ ]:
np.mean(((predicted_values > 0.5)*1).reshape(-1) == y_tgt_test)

0.9791327233989222

In [ ]:
y_aux1_test[21:40]

array([[0, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int64)

In [ ]:
pd.crosstab(y_tgt_test, 
            ((predicted_values>0.5)*1).reshape(-1))

col_0,0,1
row_0,,
0,15063,89
1,244,562


In [ ]:
(15063+562)/(14213+1038+177+530)

0.9791327233989222

In [ ]:
(14993+491)/(14213+1038+177+530)

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
np.mean(((predicted_values > 0.5)*1).reshape(-1) == y_tgt_test)

0.9562601829803233

In [ ]:
np.mean((predicted_values > 0.5)*1 == y_tgt_test)

0.8443108846271029

In [ ]:
import keras
model_Vanilla = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic/model_Vanilla_0', compile=False)
predicted_values = model_Vanilla.predict(X_test_tokens)
np.mean(((predicted_values > 0.5)*1).reshape(-1) == y_tgt_test)

0.9578894598320592

In [ ]:
import keras
model_Aux_ce2 = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic/model_Aux_ce2_1', compile=False)

In [ ]:
model_Aux2.get_layer(index = -2).get_weights()[1]

(256,)

In [ ]:
pd.DataFrame(X_train_tokens[:,0:5]).describe()

,0,1,2,3,4
count,143613.0,143613.000000,143613.000000,143613.000000,143613.000000
mean,0.0,1.246411,1.044594,0.916370,0.917893
std,0.0,1.112982,1.050798,1.129044,1.081726
min,0.0,0.000000,0.000000,0.000000,0.000000
25%,0.0,0.000000,0.000000,0.000000,0.000000
50%,0.0,0.913119,0.988007,0.000000,1.091583
75%,0.0,2.178971,1.672841,1.905578,1.848210
max,0.0,6.493616,6.339151,7.284314,7.060423


In [ ]:
extraction1 = Model(inputs=model_Aux1.inputs, outputs=model_Aux1.layers[-2].output)
extracted_features = extraction1.predict(X_test_tokens)
extracted_features = scaler1.transform(extracted_features)
predicted_values3 = model_transf1.predict(np.hstack((X_test_tokens, extracted_features)))
np.mean(((predicted_values3 > 0.5)*1).reshape(-1) == y_tgt_test)

0.9538162677027197

In [ ]:
np.hstack((X_test_tokens, extracted_features)).shape

(15958, 5256)

In [ ]:
max_length

1411

# Evaluation

In [ ]:
import keras 
from keras.models import Sequential, Model, Input
from sklearn import metrics
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

scores = {}

for fold_no in np.arange(1):
  X_train, X_test, y_aux1_train, y_aux1_test = split_data(X, y_aux_agg, fold_no)
  X_train, X_test, y_tgt_train, y_tgt_test = split_data(X, y_tgt, fold_no)
  y_aux1_train = np.array(y_aux1_train, dtype='float32')

  vocab_size = 5000

  tokenizer_obj = Tokenizer(num_words = vocab_size)
  tokenizer_obj.fit_on_texts(X_train)

  max_length = max([len(s.split()) for s in X])

  X_train_tokens = tokenizer_obj.texts_to_matrix(X_train, mode='tfidf')
  X_test_tokens = tokenizer_obj.texts_to_matrix(X_test, mode='tfidf')

  # model_Vanilla = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_Vanilla_'+str(fold_no))
  # model_linear = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_Vanilla_linear_'+str(fold_no))
  # model_joint = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_Vanilla_joint_'+str(fold_no))
  model_Aux1 = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic_SGD3/model_Aux1_'+str(fold_no), compile=False)
  model_transf1 = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic_SGD3/model_transf1_'+str(fold_no))
  model_transf_linear1 = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic_SGD3/model_transf_linear1_'+str(fold_no))
  # model_Aux_ce1 = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_Vanilla_joint_'+str(fold_no), compile=False)
  # model_transf_ce1 = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_transf_ce1_revised_'+str(fold_no))
  # model_transf_ce_linear1 = keras.models.load_model('/content/drive/MyDrive/Transfer_learning/Toxic_SGD/model_transf_ce_linear1_revised_'+str(fold_no))


  # predicted_linear = model_linear.predict(X_test_tokens)
  # predicted_Vanilla = model_Vanilla.predict(X_test_tokens)
  # predicted_joint = model_joint.predict(X_test_tokens)[:,0]
  
  extraction = Model(inputs=model_Aux1.inputs, outputs=model_Aux1.layers[-2].output)
  predicted_extract = extraction.predict(X_test_tokens)
  predicted_transf_linear = model_transf_linear1.predict(np.hstack((X_test_tokens, predicted_extract)))
  predicted_transf = model_transf1.predict(np.hstack((X_test_tokens, predicted_extract)))
  # extraction_ce = Model(inputs=model_Aux_ce1.inputs, outputs=model_Aux_ce1.layers[-2].output)
  # predicted_extract_ce = extraction_ce.predict(X_test_tokens)
  # predicted_transf_ce_linear = model_transf_ce_linear1.predict(np.hstack((X_test_tokens, predicted_extract_ce)))
  # predicted_transf_ce = model_transf_ce1.predict(np.hstack((X_test_tokens, predicted_extract_ce)))
  
  # auc_linear = metrics.roc_auc_score(y_tgt_test, (predicted_linear).reshape(-1))
  # auc_Vanilla = metrics.roc_auc_score(y_tgt_test, (predicted_Vanilla).reshape(-1))
  # auc_joint = metrics.roc_auc_score(y_tgt_test, (predicted_joint).reshape(-1))
  auc_transf = metrics.roc_auc_score(y_tgt_test, (predicted_transf).reshape(-1))
  auc_transf_linear = metrics.roc_auc_score(y_tgt_test, (predicted_transf_linear).reshape(-1))
  # auc_transf_ce = metrics.roc_auc_score(y_tgt_test, (predicted_transf_ce).reshape(-1))
  # auc_transf_ce_linear = metrics.roc_auc_score(y_tgt_test, (predicted_transf_ce_linear).reshape(-1))

  # acc_linear = np.mean(((predicted_linear > 0.5) *1).reshape(-1) == y_tgt_test)
  # acc_Vanilla = np.mean(((predicted_Vanilla > 0.5) *1).reshape(-1) == y_tgt_test)
  # acc_joint = np.mean(((predicted_joint > 0.5) *1).reshape(-1) == y_tgt_test)
  acc_transf_linear = np.mean(((predicted_transf_linear > 0.5) *1).reshape(-1) == y_tgt_test)
  acc_transf = np.mean(((predicted_transf > 0.5) *1).reshape(-1) == y_tgt_test)
  # acc_transf_ce_linear = np.mean(((predicted_transf_ce_linear > 0.5) *1).reshape(-1) == y_tgt_test)
  # acc_transf_ce = np.mean(((predicted_transf_ce > 0.5) *1).reshape(-1) == y_tgt_test)

  # f1_linear = metrics.f1_score(y_tgt_test, ((predicted_linear>0.5) *1).reshape(-1))
  # f1_Vanilla = metrics.f1_score(y_tgt_test, ((predicted_Vanilla>0.5) *1).reshape(-1))
  # f1_joint = metrics.f1_score(y_tgt_test, ((predicted_joint>0.5) *1).reshape(-1))
  f1_transf_linear = metrics.f1_score(y_tgt_test, ((predicted_transf_linear>0.5)*1).reshape(-1))
  f1_transf = metrics.f1_score(y_tgt_test, ((predicted_transf>0.5)*1).reshape(-1))
  # f1_transf_ce_linear = metrics.f1_score(y_tgt_test, ((predicted_transf_ce_linear>0.5)*1).reshape(-1))
  # f1_transf_ce = metrics.f1_score(y_tgt_test, ((predicted_transf_ce>0.5)*1).reshape(-1))

  scores[fold_no] = np.array([auc_transf_linear, auc_transf,
                              acc_transf_linear, acc_transf, 
                              f1_transf_linear, f1_transf])
  

  # scores[fold_no] = np.array([auc_linear, auc_Vanilla, auc_joint, auc_transf_ce_linear, auc_transf_linear, auc_transf_ce, auc_transf,  
  #                             acc_linear, acc_Vanilla, acc_joint, acc_transf_ce_linear, acc_transf_linear, acc_transf_ce, acc_transf,  
  #                             f1_linear, f1_Vanilla, f1_joint, f1_transf_ce_linear, f1_transf_linear, f1_transf_ce, f1_transf])
  # del X_train, X_test, y_aux1_train, y_aux1_test, y_tgt_train, y_tgt_test, tokenizer_obj, X_train_tokens, X_test_tokens

scores_final = np.vstack(scores.values())

scores_final = pd.DataFrame(scores_final, columns = ['auc_Proposed2', 'auc_Proposed4',
                                                     'acc_Proposed2', 'acc_Proposed4',
                                                     'f1_Proposed2', 'f1_Proposed4'])

# scores_final = pd.DataFrame(scores_final, columns = ['auc_Baseline1','auc_Baseline2', 'auc_Baseline3', 'auc_Proposed1', 'auc_Proposed2', 'auc_Proposed3', 'auc_Proposed4',
#                                                      'acc_Baseline1','acc_Baseline2', 'acc_Baseline3', 'acc_Proposed1', 'acc_Proposed2','acc_Proposed3', 'acc_Proposed4',
#                                                      'f1_Baseline1','f1_Baseline2', 'f1_Baseline3', 'f1_Proposed1','f1_Proposed2','f1_Proposed3', 'f1_Proposed4'])
# scores_final.to_csv('/content/drive/MyDrive/Transfer_learning/Toxic_SGD/scores_final.csv')
# scores_final.to_csv('/content/drive/MyDrive/Transfer_learning/Toxic_SGD2/scores_final_revised.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [ ]:
scores_final

,auc_Proposed2,auc_Proposed4,acc_Proposed2,acc_Proposed4,f1_Proposed2,f1_Proposed4
0,0.95886,0.958068,0.956761,0.956574,0.739229,0.743997


In [ ]:
scores_final

,auc_Proposed2,auc_Proposed4,acc_Proposed2,acc_Proposed4,f1_Proposed2,f1_Proposed4
0,0.958291,0.957172,0.958328,0.958892,0.757211,0.765882


In [ ]:
np.mean(scores_final, axis=0)

auc_Proposed3    0.958451
auc_Proposed4    0.961254
acc_Proposed3    0.958207
acc_Proposed4    0.958790
f1_Proposed3     0.765647
f1_Proposed4     0.770609
dtype: float64

In [ ]:
scores_final

array([[0.95481227, 0.94723025, 0.95051352, 0.95461274, 0.94982207,
        0.95788946, 0.95381627, 0.95745081, 0.95644818, 0.95456824,
        0.75275938, 0.7291437 , 0.75064267, 0.74363703, 0.73817263],
       [0.95965249, 0.95846067, 0.95489745, 0.95310128, 0.95322425,
        0.95882685, 0.95826283, 0.95782415, 0.95744814, 0.95563076,
        0.76710386, 0.76923077, 0.76591304, 0.76066267, 0.75484765],
       [0.95381839, 0.95418712, 0.95280688, 0.9512541 , 0.95349389,
        0.95776148, 0.95801216, 0.9571348 , 0.95600677, 0.95732281,
        0.74888227, 0.75421864, 0.75342466, 0.74095941, 0.75263349],
       [0.95621471, 0.95110088, 0.94812901, 0.9516515 , 0.95483011,
        0.95726014, 0.95506674, 0.95481607, 0.95481607, 0.95575609,
        0.74852507, 0.73898799, 0.73848386, 0.73867343, 0.74438812],
       [0.95675294, 0.95534823, 0.95726956, 0.95639186, 0.95615949,
        0.95901485, 0.9595162 , 0.95644545, 0.95694679, 0.95625744,
        0.76096491, 0.76319648, 0.74625776, 

In [ ]:
scores

{0: array([0.95481227, 0.94723025, 0.95051352, 0.95461274, 0.94982207,
        0.95788946, 0.95381627, 0.95745081, 0.95644818, 0.95456824,
        0.75275938, 0.7291437 , 0.75064267, 0.74363703, 0.73817263])}